## The records to read graphs from different popular scRNA-seq data analysis pipelines

There are various data structures to save the graphs in R, while we just need the sparse matrix of the adjacent matrix as the input of *HGC*. And we provide some methods to transfer the graph to the sparse matrix. This guide will be maintained and the advice about graphs in R packages unincluded is welcomed.

### Loading the demo data

Some pipelines may need a gene expression matrix as input, here we use the same demo dataset as in the *Plotting_Dendrogram_And_Low_Dimension_Visualization*.

In [1]:
data.d4 <- read.table("DemoData/simulated_counts_discrete_Sigma0.40.txt", header = T)
annotation.d4 <- read.table("DemoData/cell_labels_discrete_Sigma0.40.txt", header = T)

# create fake gene names for the expression matrix
data.d4 <- as.matrix(data.d4)

GeneName <- NULL
for(i in 1:3000){
  GeneName[i] <- paste("Gene",i,sep = "")
}
rownames(data.d4) <- GeneName

### Adjacent matrix

The adjacent matrix of the graph could be directly converted by the function *as* in R package *Matrix*. We need a symmetric square matrix as the input, while we do not check the symmetry inner the package. We just read the upper triangle part of the matrix. 

In [2]:
library(Matrix)

In [3]:
G = matrix(1:25, ncol = 5, nrow = 5)
G = G + t(G)
G = as(G, "dgCMatrix")
HGC::HGC.paris(G)


Call:
NA

Cluster method   : HGC 
Distance         : Node pair sampling ratio 
Number of objects: 5 


### Seurat: PCA-based preprocessings

*Seurat* is the pipeline we use in the article to do preprocessing and graph construction. The package provides its function *as.parse* to build the sparse matrix.

In [4]:
library(Seurat)
load("DemoData/datad4objects.Rdata")

In [5]:
# The preparation of Seurat object is saved in the Preparation_For_Demo_Data.ipynb.
data.d4.seuratobj <- HGC::FindClusteringTree(data.d4.seuratobj)
data.d4.seuratobj@graphs$ClusteringTree


Call:
NA

Cluster method   : HGC 
Distance         : Node pair sampling ratio 
Number of objects: 1000 


### Seurat: GLMPCA-based preprocessings

The GLMPCA has been included by package *seurat-wrappers* published in github.

In [6]:
library(Seurat)
library(SeuratWrappers)
library(glmpca)
library(scry)


Attaching package: ‘SeuratWrappers’


The following objects are masked from ‘package:Seurat’:

    ALRAChooseKPlot, ReadAlevin, RunALRA




In [7]:
set.seed(123)
# Initial processing to select variable features
m <- data.d4
devs <- scry::devianceFeatureSelection(m)
dev_ranked_genes <- rownames(data.d4)[order(devs, decreasing = TRUE)]
topdev <- head(dev_ranked_genes, 2000)

data.d4.seuratobj <- CreateSeuratObject(counts = data.d4)
data.d4.seuratobj <- RunGLMPCA(data.d4.seuratobj, features = topdev, L = 25)
data.d4.seuratobj <- FindNeighbors(data.d4.seuratobj, reduction = 'glmpca', dims = 1:25, verbose = FALSE)
data.d4.seuratobj <- HGC::FindClusteringTree(data.d4.seuratobj)
data.d4.seuratobj@graphs$ClusteringTree

Sparse matrices are not supported for minibatch='none'. Coercing to dense matrix. If this exhausts memory, consider setting minibatch to 'stochastic' or 'memoized'.




Call:
NA

Cluster method   : HGC 
Distance         : Node pair sampling ratio 
Number of objects: 1000 


### monocle3

Monocle3 is a pipeline designed from the lineage tracing of scRNA-seq datasets, and it could also finish a series of data analysis tasks like the visualization and clustering.

In [8]:
library(monocle3)

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:Matrix’:

    which


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Welcom

In [9]:
# Create monocle3 object
cds <- new_cell_data_set(expression_data = data.d4)
cds <- preprocess_cds(cds, num_dim = 100)
cds <- reduce_dimension(cds, umap.fast_sgd = F)

# The monocle3 object doesn't save the graph, instead they save the low dimension coordinates and calculate the graph from the coordinates each time.
library(RANN)
# Here we use UMAP as the instance.
reduced_dim_res <- reducedDims(cds)[["UMAP"]]

data <- reduced_dim_res
k <- 20 # default in monocle3
tmp <- RANN::nn2(data, data, k + 1, searchtype = "standard")
neighborMatrix <- tmp[[1]][, -1]
distMatrix <- tmp[[2]][, -1]

# Write the graph as sparse matrix
library(Matrix)
ncell <- nrow(neighborMatrix)
G <- Matrix::sparseMatrix(i = rep(1:ncell, k), j = as.vector(neighborMatrix), x = 1)
colnames(G) <- colnames(cds)
rownames(G) <- colnames(cds)

G <- G + t(G)

# Calculate Jaccard similarity as weights
nedge = length(G@i)/2
G.df <- as.data.frame(summary(G))
for(iter in 1:nedge){
    i <- G.df$i[iter]
    j <- G.df$j[iter]
    uni <- length(intersect(neighborMatrix[i,], neighborMatrix[j,]))
    jaccard.index <- uni/(2*k - uni)
    G[i,j] <- jaccard.index
    G[j,i] <- jaccard.index
}

HGC::HGC.paris(G)

Warning message in new_cell_data_set(expression_data = data.d4):
“Warning: gene_metadata must contain a column verbatim named 'gene_short_name' for certain functions.”
No preprocess_method specified, using preprocess_method = 'PCA'




Call:
NA

Cluster method   : HGC 
Distance         : Node pair sampling ratio 
Number of objects: 1000 


### Running HGC in MST

HGC could suit a wide range of undirected graphs. Here we show it could provide a clustering tree along the minimum spanning tree.

In [10]:
## load the PC space for datad4
load("DemoData/datad4objects.Rdata")
PCs <- data.d4.seuratobj@reductions$pca@cell.embeddings[,1:25]

require(ape)
require(Matrix)

ncell <- nrow(PCs)
D <- dist(PCs)
dmst <- ape::mst(D)
    
mst.G <- matrix(0, nrow = ncell, ncol = ncell)
mst.G[which(dmst > 0)] = 1
mst.G <- as(mst.G, "dgCMatrix")

HGC::HGC.paris(mst.G)

Loading required package: ape




Call:
NA

Cluster method   : HGC 
Distance         : Node pair sampling ratio 
Number of objects: 1000 


### Running HGC in PMST

Perturbed minimum spanning tree (PMST) is a graph construction method based on MST. It combines a number of MSTs, each fit to a perturebed version of the dataset. Meanwhile, the coustruction of PMST is time-consuming.

Reference: https://dl.acm.org/doi/10.5555/2976040.2976069.

In [11]:
require(ape)
require(Matrix)
require(RANN)

## PMST function
PMST_Construction <- function(mat, iter = 20, r = 0.5, seed = 100){
    ncell <- nrow(mat)
    nfeature <- ncol(mat)
    D <- dist(mat)
    mst.G <- matrix(0, nrow = ncell, ncol = ncell)
    
    ## Build the knn on the original space
    k <- 3
    tmp <- RANN::nn2(mat, mat, k + 1, searchtype = "standard")
    neighborMatrix <- tmp[[1]][, -1]
    distMatrix <- tmp[[2]][, -1]
    # for each node, record the coordinate of the average of its three nearest neighbors
    neighborLocation <- mat
    for(i in 1:ncell){
        neighbor1 <- neighborMatrix[i,1]
        neighbor2 <- neighborMatrix[i,2]
        neighbor3 <- neighborMatrix[i,3]
    
        neighbor1.site <- as.vector(mat[neighbor1,])
        neighbor2.site <- as.vector(mat[neighbor2,])
        neighbor3.site <- as.vector(mat[neighbor3,])
    
        new.neighbor <- (neighbor1.site + neighbor2.site + neighbor3.site)/3
        neighborLocation[i,] <- new.neighbor
    }
    
    set.seed(seed)
    ## Perturbation
    for(i in 1:iter){
        new.mat1 <- mat
        new.mat2 <- neighborLocation
        
        random.r1 <- runif(n = ncell, min = 0, max = r)
        random.r2 <- rep(1, ncell) - random.r1
        
        new.mat <- new.mat1*random.r1 + new.mat2*random.r2
        new.D <- dist(new.mat)
        new.mst <- ape::mst(new.D)
        
        new.mst.G <- matrix(0, nrow = ncell, ncol = ncell)
        new.mst.G[which(new.mst > 0)] = 1
        
        mst.G <- mst.G + new.mst.G
    }
    
    return(mst.G)
}

In [12]:
pmst.G <- PMST_Construction(mat = PCs)
pmst.G <- as(pmst.G, "dgCMatrix")

HGC::HGC.paris(pmst.G)


Call:
NA

Cluster method   : HGC 
Distance         : Node pair sampling ratio 
Number of objects: 1000 
